In [87]:
import sys
print(sys.version, sys.platform, sys.executable)

3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)] win32 C:\Users\varun\conda\envs\test_env\python.exe


In [88]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from matplotlib import pyplot as plt


import sys
sys.path.append("../")
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import Markdown, display

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as SVM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric


The CompasDataset expects you to specify the label (target) variable, by default it is 'two_year_recid'

aif360.datasets.CompasDataset(label_name='two_year_recid', favorable_classes=[0], protected_attribute_names=['sex', 'race'], privileged_classes=[['Female'], ['Caucasian']], instance_weights_name=None, categorical_features=['age_cat', 'c_charge_degree', 'c_charge_desc'], features_to_keep=['sex', 'age', 'age_cat', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'c_charge_desc', 'two_year_recid'], features_to_drop=[], na_values=[], custom_preprocessing=<function default_preprocessing>, metadata={'label_maps': [{0.0: 'No recid.', 1.0: 'Did recid.'}], 'protected_attribute_maps': [{0.0: 'Male', 1.0: 'Female'}, {0.0: 'Not Caucasian', 1.0: 'Caucasian'}]})

In [89]:
protected = 'race'
cp = CompasDataset(
    label_name='decile_score',
    protected_attribute_names=[protected],
    privileged_classes = [['Caucasian']], 
    categorical_features=['sex','age_cat',  'c_charge_degree', 'score_text'],
    features_to_keep=['sex', 'age', 'age_cat', 'race', 'score_text','v_decile_score', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'two_year_recid'])

### Train and Test split

In [90]:
#Splitting to Train and Test- to be used through all models

train, test = cp.split([0.7], shuffle=True)

### ------------Model 1: Logistic Regression before employing Disparate Impact Remover-------------

In [91]:
#Converting train and test to Pandas

data_train=train.convert_to_dataframe()
df_train=pd.DataFrame(data_train[0])

data_test=test.convert_to_dataframe()
df_test=pd.DataFrame(data_test[0])

In [92]:
df_train.v_decile_score=df_train.v_decile_score.replace([-1,1,2,3,4,5,6,7,8,9,10],[0,0,0,0,0,0,1,1,1,1,1])
df_test.v_decile_score=df_test.v_decile_score.replace([-1,1,2,3,4,5,6,7,8,9,10],[0,0,0,0,0,0,1,1,1,1,1])

In [95]:
#Shape of Train and Test 

display(df_train.shape)
display(df_test.shape)

(4320, 19)

(1852, 19)

In [103]:
#converting from float to int
#Train

df_train['age']=df_train['age'].astype('int64')
df_train['race']=df_train[ 'race'].astype('int64')
df_train['juv_fel_count']=df_train['juv_fel_count'].astype('int64')
df_train['v_decile_score']=df_train['v_decile_score'].astype('int64')
df_train['juv_misd_count']=df_train['juv_misd_count'].astype('int64')
df_train['juv_other_count']=df_train['juv_other_count'].astype('int64')
df_train['priors_count']=df_train['priors_count'].astype('int64')
df_train['sex=Female']=df_train['sex=Female'].astype('int64')
df_train['sex=Male']=df_train['sex=Male'].astype('int64')
df_train['age_cat=25 - 45']=df_train['age_cat=25 - 45'].astype('int64')
df_train['age_cat=Greater than 45']=df_train['age_cat=Greater than 45'].astype('int64')
df_train['age_cat=Less than 25']=df_train['age_cat=Less than 25'].astype('int64')
df_train['two_year_recid']=df_train['two_year_recid'].astype('int64')


In [104]:
#converting from float to int
#Test

df_test['age']=df_test['age'].astype('int64')
df_test['race']=df_test[ 'race'].astype('int64')
df_test['juv_fel_count']=df_test['juv_fel_count'].astype('int64')
df_test['v_decile_score']=df_test['v_decile_score'].astype('int64')
df_test['juv_misd_count']=df_test['juv_misd_count'].astype('int64')
df_test['juv_other_count']=df_test['juv_other_count'].astype('int64')
df_test['priors_count']=df_test['priors_count'].astype('int64')
df_test['sex=Female']=df_test['sex=Female'].astype('int64')
df_test['sex=Male']=df_test['sex=Male'].astype('int64')
df_test['age_cat=25 - 45']=df_test['age_cat=25 - 45'].astype('int64')
df_test['age_cat=Greater than 45']=df_test['age_cat=Greater than 45'].astype('int64')
df_test['age_cat=Less than 25']=df_test['age_cat=Less than 25'].astype('int64')
df_test['two_year_recid']=df_test['two_year_recid'].astype('int64')

In [105]:
#Train
xTrain=df_train.drop(['v_decile_score', 'two_year_recid'], axis=1)
yTrain=df_train['v_decile_score']

#Test
xTest=df_test.drop([ 'v_decile_score', 'two_year_recid'], axis=1)
yTest=df_test['v_decile_score']

In [106]:
#model
classifier=LogisticRegression()
classifier.fit(xTrain, yTrain)

C:\Users\varun\conda\envs\test_env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [108]:
#predict
yPred=classifier.predict(xTest)

In [109]:
model_1=df_test.copy() #Storing test to a new dataset

In [110]:
model_1['true_risk'] = model_1['two_year_recid'] #ground truth

In [111]:
model_1['pred_risk']=yPred #predicted decile score binary #appending the predicted labels to the final dataset

Accuracy


In [119]:
# Prediction Accuracy

accuracy_score(model_1['true_risk'], model_1['pred_risk'])

0.6506479481641468

In [120]:
# Prediction Accuracy for Caucasians
c1=model_1[model_1['race']==1]

accuracy_score(c1['true_risk'], c1['pred_risk'])

0.6823899371069182

In [121]:
classifier.score(xTest,yTest) #model accuracy

0.8623110151187905

In [122]:
classifier.score(xTest[xTest["race"] == 1],yTest[xTest["race"] == 1]) #Caucasians

0.8915094339622641

In [123]:
classifier.score(xTest[xTest["race"] == 0],yTest[xTest["race"] == 0])

0.8470394736842105

### ----------------Model 2: Logistic Regression after employing Disparate Impact Remover---------------


In [124]:
display(Markdown("#### Training Dataset shape"))
print(train.features.shape)

#### Training Dataset shape

(4320, 18)


In [125]:
display(Markdown("#### Protected attribute names"))
print(train.protected_attribute_names)

#### Protected attribute names

['race']


In [126]:
display(Markdown("#### Label names"))
print(train.label_names)

#### Label names

['decile_score']


In [127]:
display(Markdown("#### Dataset feature names"))
print(train.feature_names)

#### Dataset feature names

['age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'v_decile_score', 'two_year_recid', 'sex=Female', 'sex=Male', 'age_cat=25 - 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M', 'score_text=High', 'score_text=Low', 'score_text=Medium']


In [128]:
#applying Disparate Impact Remover
di = DisparateImpactRemover(repair_level=1.0)
train_repd = di.fit_transform(train)
test_repd = di.fit_transform(test)




In [129]:
#Convert to Pandas after applying Disparate Impact Remover

#Train
data1=train_repd.convert_to_dataframe()
train_di=pd.DataFrame(data1[0])


#Test
data2=test_repd.convert_to_dataframe()
test_di=pd.DataFrame(data2[0])

In [130]:
display(train_di.shape)
display(test_di.shape)

(4320, 19)

(1852, 19)

In [131]:
#converting from float to int
#Train

train_di['age']=train_di['age'].astype('int64')
train_di['race']=train_di[ 'race'].astype('int64')
train_di['juv_fel_count']=train_di['juv_fel_count'].astype('int64')
train_di['v_decile_score']=train_di['v_decile_score'].astype('int64')
train_di['juv_misd_count']=train_di['juv_misd_count'].astype('int64')
train_di['juv_other_count']=train_di['juv_other_count'].astype('int64')
train_di['priors_count']=train_di['priors_count'].astype('int64')
train_di['sex=Female']=train_di['sex=Female'].astype('int64')
train_di['sex=Male']=train_di['sex=Male'].astype('int64')
train_di['age_cat=25 - 45']=train_di['age_cat=25 - 45'].astype('int64')
train_di['age_cat=Greater than 45']=train_di['age_cat=Greater than 45'].astype('int64')
train_di['age_cat=Less than 25']=train_di['age_cat=Less than 25'].astype('int64')
train_di['two_year_recid']=train_di['two_year_recid'].astype('int64')

In [132]:
#Test
test_di['age']=test_di['age'].astype('int64')
test_di['race']=test_di[ 'race'].astype('int64')
test_di['juv_fel_count']=test_di['juv_fel_count'].astype('int64')
test_di['v_decile_score']=test_di['v_decile_score'].astype('int64')
test_di['juv_misd_count']=test_di['juv_misd_count'].astype('int64')
test_di['juv_other_count']=test_di['juv_other_count'].astype('int64')
test_di['priors_count']=test_di['priors_count'].astype('int64')
test_di['sex=Female']=test_di['sex=Female'].astype('int64')
test_di['sex=Male']=test_di['sex=Male'].astype('int64')
test_di['age_cat=25 - 45']=test_di['age_cat=25 - 45'].astype('int64')
test_di['age_cat=Greater than 45']=test_di['age_cat=Greater than 45'].astype('int64')
test_di['age_cat=Less than 25']=test_di['age_cat=Less than 25'].astype('int64')
test_di['two_year_recid']=test_di['two_year_recid'].astype('int64')

In [133]:
train_di.v_decile_score=train_di.v_decile_score.replace([-1,1,2,3,4,5,6,7,8,9,10],[0,0,0,0,0,0,1,1,1,1,1])
test_di.v_decile_score=test_di.v_decile_score.replace([-1,1,2,3,4,5,6,7,8,9,10],[0,0,0,0,0,0,1,1,1,1,1])

In [134]:
#Creating features and labels

#Train
xTrain_di=train_di.drop([ 'decile_score', 'two_year_recid','v_decile_score'], axis=1)
yTrain_di=train_di['v_decile_score']

#Test
xTest_di=test_di.drop([ 'decile_score', 'two_year_recid','v_decile_score'], axis=1)
yTest_di=test_di['v_decile_score']


In [135]:
#model
classifier=LogisticRegression()
classifier.fit(xTrain_di, yTrain_di)

C:\Users\varun\conda\envs\test_env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [136]:
#predict
yPred=classifier.predict(xTest_di)

In [137]:
model_2=df_test.copy() #Storing test to a new dataset
model_2['true_risk'] = model_2['two_year_recid'] #ground truth
model_2['pred_risk']=yPred #predicted decile score binary #appending the predicted labels to the final dataset

In [138]:
model_2.head(7)

,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,v_decile_score,two_year_recid,sex=Female,sex=Male,...,age_cat=Greater than 45,age_cat=Less than 25,c_charge_degree=F,c_charge_degree=M,score_text=High,score_text=Low,score_text=Medium,decile_score,true_risk,pred_risk
10766,36,0,0,0,0,9,1,1,0,1,...,0,0,1.0,0.0,0.0,0.0,1.0,0.0,1,0
3830,26,0,0,0,0,0,0,0,1,0,...,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0,0
9746,23,0,0,0,0,0,1,0,0,1,...,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0,1
4996,27,1,0,0,0,1,0,0,0,1,...,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0,0
7206,37,0,0,0,0,6,0,1,0,1,...,0,0,1.0,0.0,0.0,1.0,0.0,0.0,1,0
9724,64,0,0,0,0,0,0,0,1,0,...,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0,0
10553,28,0,0,0,0,13,0,1,0,1,...,0,0,1.0,0.0,1.0,0.0,0.0,0.0,1,1


In [139]:

accuracy_score(model_2['true_risk'], model_2['pred_risk'])

0.6506479481641468

In [140]:
classifier.score(xTest_di,yTest_di) #model accuracy

0.8623110151187905

In [141]:
classifier.score(xTest_di[xTest_di["race"] == 1],yTest_di[xTest["race"] == 1]) #Caucasians

0.8915094339622641

In [142]:
classifier.score(xTest_di[xTest_di["race"] == 0],yTest_di[xTest["race"] == 0])

0.8470394736842105